Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (200000, 28, 28), (200000,))
('Validation set', (10000, 28, 28), (10000,))
('Test set', (10000, 28, 28), (10000,))


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (200000, 28, 28, 1), (200000, 10))
('Validation set', (10000, 28, 28, 1), (10000, 10))
('Test set', (10000, 28, 28, 1), (10000, 10))


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.069957
Minibatch accuracy: 0.0%
Validation accuracy: 10.4%
Minibatch loss at step 50: 2.050407
Minibatch accuracy: 25.0%
Validation accuracy: 31.4%
Minibatch loss at step 100: 0.658478
Minibatch accuracy: 87.5%
Validation accuracy: 48.1%
Minibatch loss at step 150: 1.252247
Minibatch accuracy: 62.5%
Validation accuracy: 51.9%
Minibatch loss at step 200: 0.763965
Minibatch accuracy: 75.0%
Validation accuracy: 56.8%
Minibatch loss at step 250: 1.122011
Minibatch accuracy: 62.5%
Validation accuracy: 60.2%
Minibatch loss at step 300: 0.360103
Minibatch accuracy: 87.5%
Validation accuracy: 62.4%
Minibatch loss at step 350: 0.243691
Minibatch accuracy: 93.8%
Validation accuracy: 60.7%
Minibatch loss at step 400: 0.327666
Minibatch accuracy: 87.5%
Validation accuracy: 62.5%
Minibatch loss at step 450: 0.753511
Minibatch accuracy: 75.0%
Validation accuracy: 63.0%
Minibatch loss at step 500: 0.587000
Minibatch accuracy: 87.5%
Validation accuracy: 63.9%
Mi

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [33]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    h_pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    conv = tf.nn.conv2d(h_pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    h_pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    shape = h_pool.get_shape().as_list()
    reshape = tf.reshape(h_pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [34]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.618979
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.334800
Minibatch accuracy: 18.8%
Validation accuracy: 23.1%
Minibatch loss at step 100: 1.537322
Minibatch accuracy: 43.8%
Validation accuracy: 22.9%
Minibatch loss at step 150: 0.983817
Minibatch accuracy: 68.8%
Validation accuracy: 52.7%
Minibatch loss at step 200: 1.076541
Minibatch accuracy: 56.2%
Validation accuracy: 57.5%
Minibatch loss at step 250: 1.279957
Minibatch accuracy: 56.2%
Validation accuracy: 61.0%
Minibatch loss at step 300: 0.371758
Minibatch accuracy: 87.5%
Validation accuracy: 63.1%
Minibatch loss at step 350: 0.139954
Minibatch accuracy: 100.0%
Validation accuracy: 63.9%
Minibatch loss at step 400: 0.483642
Minibatch accuracy: 81.2%
Validation accuracy: 63.5%
Minibatch loss at step 450: 0.770515
Minibatch accuracy: 75.0%
Validation accuracy: 63.7%
Minibatch loss at step 500: 0.795403
Minibatch accuracy: 75.0%
Validation accuracy: 66.8%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [147]:
# Add more conv layers
batch_size = 64
patch_size = 5
depth_1 = 32
depth_2 = 32
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth_1], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth_1]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth_1, depth_1], stddev=0.1))  
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth_1]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth_1, depth_2], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[depth_2]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [2, 2, depth_2, depth_2], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[depth_2]))
  layer5_weights = tf.Variable(tf.truncated_normal(
      [depth_2, num_hidden], stddev=0.1))
  layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer6_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer6_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  keep_prob = tf.placeholder("float")
    
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    h_pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    conv = tf.nn.conv2d(h_pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    h_pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    conv = tf.nn.conv2d(h_pool, layer3_weights, [1, 1, 1, 1], padding='VALID')
    hidden = tf.nn.relu(conv + layer3_biases)
    h_pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  
    conv = tf.nn.conv2d(h_pool, layer4_weights, [1, 1, 1, 1], padding='VALID')
    hidden = tf.nn.relu(conv + layer4_biases)    
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    h_fc1_drop = tf.nn.dropout(reshape, keep_prob)
    hidden = tf.nn.relu(tf.matmul(h_fc1_drop, layer5_weights) + layer5_biases)
    return tf.matmul(hidden, layer6_weights) + layer6_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  C = 0.005
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + \
    C * (tf.nn.l2_loss(layer6_weights) + tf.nn.l2_loss(layer5_weights))
    
  # Optimizer.
  global_step = tf.Variable(0, trainable=False) # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.05, global_step, 10000, 0.97)  
#   optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [148]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
#     rand_idx = np.random.choice(train_dataset.shape[0], batch_size, replace=False) 
#     batch_data = train_dataset[rand_idx, :, :, :]
#     batch_labels = train_labels[rand_idx, :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(feed_dict={keep_prob: 1}), valid_labels, ))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(feed_dict={keep_prob: 1}), test_labels))

Initialized
Minibatch loss at step 0: 3.116156
Minibatch accuracy: 10.9%
Validation accuracy: 10.0%
Minibatch loss at step 500: 0.953602
Minibatch accuracy: 75.0%
Validation accuracy: 61.8%
Minibatch loss at step 1000: 0.754872
Minibatch accuracy: 79.7%
Validation accuracy: 65.3%
Minibatch loss at step 1500: 0.507935
Minibatch accuracy: 84.4%
Validation accuracy: 72.1%
Minibatch loss at step 2000: 0.397281
Minibatch accuracy: 90.6%
Validation accuracy: 73.5%
Minibatch loss at step 2500: 0.615623
Minibatch accuracy: 82.8%
Validation accuracy: 73.0%
Minibatch loss at step 3000: 0.499430
Minibatch accuracy: 84.4%
Validation accuracy: 74.4%
Minibatch loss at step 3500: 0.480713
Minibatch accuracy: 84.4%
Validation accuracy: 75.3%
Minibatch loss at step 4000: 0.697069
Minibatch accuracy: 81.2%
Validation accuracy: 75.3%
Minibatch loss at step 4500: 0.671578
Minibatch accuracy: 81.2%
Validation accuracy: 76.0%
Minibatch loss at step 5000: 0.404246
Minibatch accuracy: 87.5%
Validation accurac

In [119]:
# Add more conv layers
batch_size = 64
patch_size = 5
depth_1 = 16
depth_2 = 64
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth_1], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth_1]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth_1, depth_1], stddev=0.1))  
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth_1]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth_1, depth_2], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[depth_2]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [2 * 2 * depth_2, num_hidden], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer5_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  keep_prob = tf.placeholder("float")
    
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    h_pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    conv = tf.nn.conv2d(h_pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    h_pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    conv = tf.nn.conv2d(h_pool, layer3_weights, [1, 1, 1, 1], padding='VALID')
    hidden = tf.nn.relu(conv + layer3_biases)
    h_pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  
    shape = h_pool.get_shape().as_list()
    reshape = tf.reshape(h_pool, [shape[0], shape[1] * shape[2] * shape[3]])
    h_fc1_drop = tf.nn.dropout(reshape, keep_prob)
    hidden = tf.nn.relu(tf.matmul(h_fc1_drop, layer4_weights) + layer4_biases)
    return tf.matmul(hidden, layer5_weights) + layer5_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  C = 0.005
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + \
    C * (tf.nn.l2_loss(layer1_weights) + tf.nn.l2_loss(layer2_weights) + tf.nn.l2_loss(layer3_weights) + 
         tf.nn.l2_loss(layer4_weights) + tf.nn.l2_loss(layer5_weights))
    
  # Optimizer.
  global_step = tf.Variable(0, trainable=False) # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.05, global_step, 10000, 0.97)  
#   optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [120]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(feed_dict={keep_prob: 1}), valid_labels, ))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(feed_dict={keep_prob: 1}), test_labels))

Initialized
Minibatch loss at step 0: 4.907928
Minibatch accuracy: 14.1%
Validation accuracy: 10.0%
Minibatch loss at step 500: 1.471506
Minibatch accuracy: 75.0%
Validation accuracy: 67.2%
Minibatch loss at step 1000: 1.353490
Minibatch accuracy: 82.8%
Validation accuracy: 70.9%
Minibatch loss at step 1500: 1.035911
Minibatch accuracy: 85.9%
Validation accuracy: 73.3%
Minibatch loss at step 2000: 0.846740
Minibatch accuracy: 87.5%
Validation accuracy: 74.8%
Minibatch loss at step 2500: 0.873334
Minibatch accuracy: 85.9%
Validation accuracy: 75.9%
Minibatch loss at step 3000: 0.684703
Minibatch accuracy: 90.6%
Validation accuracy: 76.3%
Minibatch loss at step 3500: 0.640610
Minibatch accuracy: 90.6%
Validation accuracy: 76.8%
Minibatch loss at step 4000: 0.766071
Minibatch accuracy: 79.7%
Validation accuracy: 76.9%
Minibatch loss at step 4500: 0.728170
Minibatch accuracy: 84.4%
Validation accuracy: 75.9%
Minibatch loss at step 5000: 0.539854
Minibatch accuracy: 89.1%
Validation accurac